In [ ]:
!git clone https://github.com/serp-ai/bark-with-voice-clone.git

Cloning into 'bark-with-voice-clone'...
remote: Enumerating objects: 559, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 559 (delta 7), reused 15 (delta 5), pack-reused 538
Receiving objects: 100% (559/559), 1.45 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (237/237), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd bark-with-voice-clone/

/content/bark-with-voice-clone


For downloading google drive folder

In [ ]:
# !gdown --folder https://drive.google.com/drive/folders/1161kR2CWV0JqdnnnQTyBz2SBSZjYH7KN?usp=sharing.. -O /content/datasets


In [ ]:
# https://drive.google.com/drive/folders/1QOFKpRmXJnF2Xbl9am1tTmqOBdIM0a33?usp=sharing

In [ ]:
!pip install .

Processing /content/bark-with-voice-clone
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.7 MB/s eta 0:00:00
  Created wheel 

In [ ]:
%ls

bark/                   generate.ipynb  README.md            train_coarse.ipynb
build/                  hubert/         rvc_infer.py         train_fine.ipynb
clone_voice.ipynb       LICENSE.md      rvc_test.ipynb       train_semantic.ipynb
datasets/               model-card.md   setup.py             utils/
FUNDING.yml             notebooks/      suno_bark.egg-info/
generate_chunked.ipynb  pyproject.toml  test_models.ipynb


In [ ]:
# !pip install audiolm_pytorch

In [ ]:
!pip install encodec
!pip install accelerate
!pip install transformers
!pip install huggingface_hub
!pip install diffusers
!pip install funcy
!pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 23.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11289839 sha256=802d1b1d772daafe7bebf69c604fc7cbfcbf72b8b0740399929daf2d051ad33b
  Stored in directory: /root/.cache/pip/wh

In [ ]:
import torch
import torch.nn as nn
import os
import re
import gc
import json
import math
import hashlib
import numpy as np
import logging
import torchaudio
from tqdm.auto import tqdm
import torch.nn.functional as F
from encodec.utils import convert_audio
from accelerate import Accelerator
from accelerate.utils import set_seed
from transformers import BertTokenizer
from huggingface_hub import hf_hub_download
from packaging import version
from diffusers.optimization import get_scheduler

from utils.bitsandbytes import BitsAndBytesConfig, importlib_metadata, get_keys_to_not_convert, replace_with_bnb_linear, set_module_quantized_tensor_to_device
from utils.lora import convert_linear_layer_to_lora, only_optimize_lora_parameters, convert_lora_to_linear_layer
from bark.model import GPTConfig, GPT
from bark.model_fine import FineGPT, FineGPTConfig

In [ ]:
import torch
print(torch.__version__)


2.0.1+cu118


In [ ]:
!wget https://huggingface.co/junwchina/bark-voice-cloning-japanese-HuBERT-quantizer/resolve/main/japanese-HuBERT-quantizer_24_epoch.pth

--2023-10-16 09:09:34--  https://huggingface.co/junwchina/bark-voice-cloning-japanese-HuBERT-quantizer/resolve/main/japanese-HuBERT-quantizer_24_epoch.pth
Resolving huggingface.co (huggingface.co)... 108.138.94.27, 108.138.94.52, 108.138.94.97, ...
Connecting to huggingface.co (huggingface.co)|108.138.94.27|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/ee/5b/ee5b2068e250297a0993f6c3c1cb0763e7789f79a3bf96228362e1168bf1353b/07d162b3b42e8d51276ed6767105bdbc4e0b6c856a81c3f8b510fcafc34d76a6?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27japanese-HuBERT-quantizer_24_epoch.pth%3B+filename%3D%22japanese-HuBERT-quantizer_24_epoch.pth%22%3B&Expires=1697706575&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NzcwNjU3NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9lZS81Yi9lZTViMjA2OGUyNTAyOTdhMDk5M2Y2YzNjMWNiMDc2M2U3Nzg5Zjc5YTNiZjk2MjI4MzYyZTExNj

In [ ]:
# !wget https://huggingface.co/rinna/japanese-hubert-base/resolve/main/fairseq/model.pt

In [ ]:
train_batch_size = 3
eval_batch_size = 3
grad_accum = 3
ckpt_path = 'models/text_2.pt'
model_type = "text"
# dataset_path = 'datasets/joe_biden_state_of_union/'
dataset_path = '/content/drive/MyDrive/jonathan-folder/OCT11/preprocess/'

project_dir = 'logs/'
log_with = 'wandb'
hubert_path = 'data/models/hubert/hubert.pt'
# hubert_path = '/content/bark-with-voice-clone/model.pt'

hubert_tokenizer_path = '/content/bark-with-voice-clone/japanese-HuBERT-quantizer_24_epoch.pth'

output_dir = 'semantic_output/'
resume_from_checkpoint = None

checkpointing_steps = 1000

mixed_precision = 'fp16'
bits = 16 #4 4 and 8 bit are a work in progress
compute_dtype = torch.bfloat16
double_quant = True
quant_type = 'nf4'

lora_dim = 64
lora_scaling = 1
lora_dropout = 0.1
lora_module_name = 'transformer.h'
optimize_lora_params_only = False

learning_rate = 1e-4
scale_lr = False
use_8bit_adam = False
adam_beta1 = 0.9
adam_beta2 = 0.999
adam_epsilon = 1e-8
weight_decay = 0.01

llm_int8_skip_modules = None
keep_in_fp32_modules = ['lm_head']

lr_scheduler_type = 'linear'
lr_warmup_steps = 60
num_train_epochs = 15
max_train_steps = None
max_grad_norm = 1.0

seed = 741

In [ ]:
CONTEXT_WINDOW_SIZE = 1024

MAX_TEXT_LEN = 256

SEMANTIC_RATE_HZ = 49.9
SEMANTIC_VOCAB_SIZE = 10_000

TEXT_ENCODING_OFFSET = 10_048
SEMANTIC_PAD_TOKEN = 10_000
TEXT_PAD_TOKEN = 129_595
SEMANTIC_INFER_TOKEN = 129_599

MAX_SEMANTIC_LEN = 511

SAMPLE_RATE = 24_000
CHANNELS = 1

logger = logging.getLogger(__name__)


USE_SMALL_MODELS = os.environ.get("SERP_USE_SMALL_MODELS", False)

default_cache_dir = os.path.join(os.path.expanduser("~"), ".cache")
CACHE_DIR = os.path.join(os.getenv("XDG_CACHE_HOME", default_cache_dir), "serp", "bark_v0")


def _clear_cuda_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()


def _md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


def _download(from_hf_path, file_name, to_local_path):
    to_local_path = to_local_path.replace("\\", "/")
    path = '/'.join(to_local_path.split("/")[:-1])
    os.makedirs(path, exist_ok=True)
    hf_hub_download(repo_id=from_hf_path, filename=file_name, local_dir=path)
    os.replace(os.path.join(path, file_name), to_local_path)


def _tokenize(tokenizer, text):
    return tokenizer.encode(text, add_special_tokens=False)


def _detokenize(tokenizer, enc_text):
    return tokenizer.decode(enc_text)


def _normalize_whitespace(text):
    return re.sub(r"\s+", " ", text).strip()


REMOTE_MODEL_PATHS = {
    "text_small": {
        "repo_id": "suno/bark",
        "file_name": "text.pt",
        "checksum": "b3e42bcbab23b688355cd44128c4cdd3",
    },
    "coarse_small": {
        "repo_id": "suno/bark",
        "file_name": "coarse.pt",
        "checksum": "5fe964825e3b0321f9d5f3857b89194d",
    },
    "fine_small": {
        "repo_id": "suno/bark",
        "file_name": "fine.pt",
        "checksum": "5428d1befe05be2ba32195496e58dc90",
    },
    "text": {
        "repo_id": "suno/bark",
        "file_name": "text_2.pt",
        "checksum": "54afa89d65e318d4f5f80e8e8799026a",
    },
    "coarse": {
        "repo_id": "suno/bark",
        "file_name": "coarse_2.pt",
        "checksum": "8a98094e5e3a255a5c9c0ab7efe8fd28",
    },
    "fine": {
        "repo_id": "suno/bark",
        "file_name": "fine_2.pt",
        "checksum": "59d184ed44e3650774a2f0503a48a97b",
    },
}


def _load_model(ckpt_path, device, use_small=False, model_type="text"):
    if model_type == "text":
        ConfigClass = GPTConfig
        ModelClass = GPT
    elif model_type == "coarse":
        ConfigClass = GPTConfig
        ModelClass = GPT
    elif model_type == "fine":
        ConfigClass = FineGPTConfig
        ModelClass = FineGPT
    else:
        raise NotImplementedError()
    model_key = f"{model_type}_small" if use_small or USE_SMALL_MODELS else model_type
    model_info = REMOTE_MODEL_PATHS[model_key]
    if ckpt_path in [None, '']:
        ckpt_path = os.path.join(CACHE_DIR, model_info["file_name"])
    if not os.path.exists(ckpt_path):
        logger.info(f"{model_type} model not found, downloading into `{CACHE_DIR}`.")
        _download(model_info["repo_id"], model_info["file_name"], ckpt_path)
    checkpoint = torch.load(ckpt_path, map_location=device)
    # this is a hack
    model_args = checkpoint["model_args"]
    if "input_vocab_size" not in model_args:
        model_args["input_vocab_size"] = model_args["vocab_size"]
        model_args["output_vocab_size"] = model_args["vocab_size"]
        del model_args["vocab_size"]
    gptconf = ConfigClass(**checkpoint["model_args"])
    model = ModelClass(gptconf)
    state_dict = checkpoint["model"]
    # fixup checkpoint
    unwanted_prefix = "_orig_mod."
    for k, v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)
    extra_keys = set(state_dict.keys()) - set(model.state_dict().keys())
    extra_keys = set([k for k in extra_keys if not k.endswith(".attn.bias")])
    missing_keys = set(model.state_dict().keys()) - set(state_dict.keys())
    missing_keys = set([k for k in missing_keys if not k.endswith(".attn.bias")])
    if len(extra_keys) != 0:
        raise ValueError(f"extra keys found: {extra_keys}")
    if len(missing_keys) != 0:
        raise ValueError(f"missing keys: {missing_keys}")
    model.load_state_dict(state_dict, strict=False)
    n_params = model.get_num_params()
    val_loss = checkpoint["best_val_loss"].item()
    print(f"Loaded {model_type} model with {n_params} params, val_loss={val_loss:.4f}.")
    del checkpoint, state_dict
    _clear_cuda_cache()
    if model_type == "text":
        tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
        # tokenizer = BertTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking") # tokenizer model
        return model, tokenizer
    return model


def load_filepaths_and_text(filename, split="|"):
    with open(filename, encoding='utf-8', errors='ignore') as f:
        filepaths_and_text = [line.strip().split(split) for line in f]
        base = os.path.dirname(filename)
        for j in range(len(filepaths_and_text)):
            filepaths_and_text[j][0] = os.path.join(base, filepaths_and_text[j][0])
    return filepaths_and_text

class TtsDataset(torch.utils.data.Dataset):
    def __init__(self, opt):
        self.path = os.path.dirname(opt['path'])
        self.mode = opt['mode']
        self.audiopaths_and_text = load_filepaths_and_text(os.path.join(opt['path'] , opt['mode'] + '_valid.txt'))
        self.tokenizer = opt['tokenizer']

    def __getitem__(self, index):
        audiopath_and_text = self.audiopaths_and_text[index]
        audiopath, text = audiopath_and_text[0], audiopath_and_text[1]

        input_ids = np.array(_tokenize(self.tokenizer, text)) + TEXT_ENCODING_OFFSET
        input_ids = torch.from_numpy(input_ids).long()
        tokens = np.load(audiopath.replace('.wav', '.npz').replace('wavs', 'tokens'))
        semantic_tokens = tokens['semantic']
        semantic_tokens = torch.from_numpy(semantic_tokens).long()

        return input_ids, semantic_tokens

    def __len__(self):
        return len(self.audiopaths_and_text)


class TtsCollater():
    def __init__(self):
        pass
    def __call__(self, batch):
        max_text_len = MAX_TEXT_LEN
        max_semantic_tokens_len = MAX_SEMANTIC_LEN
        texts = []
        semantic_tokens = []

        for b in batch:
            text, semantic_tokens_ = b
            text = F.pad(text, (0, max_text_len-len(text)), value=TEXT_PAD_TOKEN)
            semantic_history = torch.from_numpy(np.array([SEMANTIC_PAD_TOKEN] * 256))
            text = torch.cat([text, semantic_history, torch.tensor([SEMANTIC_INFER_TOKEN])])
            texts.append(text)
            semantic_tokens_ = semantic_tokens_[:max_semantic_tokens_len]
            semantic_tokens.append(F.pad(semantic_tokens_, (0, max_semantic_tokens_len-len(semantic_tokens_)), value=SEMANTIC_PAD_TOKEN))

        return {
            'input_ids': torch.stack(texts).contiguous(),
            'semantic_tokens': torch.stack(semantic_tokens).contiguous()
        }


accelerator = Accelerator(
    gradient_accumulation_steps=grad_accum,
    mixed_precision=mixed_precision,
    log_with=log_with,
    project_dir=project_dir,
)
device = accelerator.device

os.makedirs(output_dir, exist_ok=True)

set_seed(seed)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:382: UserWarning: `log_with=wandb` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")


In [ ]:
!pip install audiolm_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00


Setup Dataset (only need to do this once)

In [ ]:
# prin

In [ ]:
# max_duration_sec = 15.12 # the maximum allowed duration in seconds

# path = dataset_path

# # From https://github.com/gitmylo/bark-voice-cloning-HuBERT-quantizer
# from hubert.hubert_manager import HuBERTManager
# hubert_manager = HuBERTManager()
# from hubert.pre_kmeans_hubert import CustomHubert
# from hubert.customtokenizer import CustomTokenizer
# hubert_manager.make_sure_hubert_installed()
# hubert_manager.make_sure_tokenizer_installed()

# # Load the HuBERT model
# hubert_model = CustomHubert(checkpoint_path=hubert_path).to(device)
# hubert_model.eval()
# for param in hubert_model.parameters():
#     param.requires_grad = False

# # Load the CustomTokenizer model
# hubert_tokenizer = CustomTokenizer.load_from_checkpoint(hubert_tokenizer_path).to(device)  # Automatically uses the right layers

# from bark.generation import load_codec_model
# codec_model = load_codec_model(use_gpu=True)
# codec_model.eval()
# for param in codec_model.parameters():
#     param.requires_grad = False


# def get_duration(wav, sr):
#     return wav.shape[1] / sr

# valid_lines_train = []
# # convert wavs to semantic tokens
# for wav_path, txt in load_filepaths_and_text(path + 'train.txt'):
#     wav, sr = torchaudio.load(wav_path)
#     if not get_duration(wav, sr) > max_duration_sec:
#         valid_lines_train.append((wav_path, txt))
#     wav = convert_audio(wav, sr, SAMPLE_RATE, CHANNELS).to(device)

#     semantic_vectors = hubert_model.forward(wav, input_sample_hz=SAMPLE_RATE)
#     semantic_tokens = hubert_tokenizer.get_token(semantic_vectors)

#     # save semantic tokens
#     os.makedirs(os.path.join(path, 'tokens'), exist_ok=True)
#     semantic_tokens = semantic_tokens.cpu().numpy()

#     # Extract discrete codes from EnCodec
#     with torch.no_grad():
#         encoded_frames = codec_model.encode(wav.unsqueeze(0))
#     codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

#     # move codes to cpu
#     codes = codes.cpu().numpy()

#     # save tokens
#     np.savez_compressed(os.path.join(path, 'tokens', os.path.basename(wav_path).replace('.wav', '.npz')), fine=codes, coarse=codes[:2, :], semantic=semantic_tokens)

# # rewrite train.txt with valid lines
# with open(path + 'train_valid.txt', 'w', encoding='utf-8') as f:
#     for wav_path, txt in valid_lines_train:
#         wav_path = os.path.relpath(wav_path, dataset_path).replace('\\', '/')
#         f.write(f'{wav_path}|{txt}\n')

# valid_lines_valid = []
# for wav_path, txt in load_filepaths_and_text(path + 'valid.txt'):
#     wav, sr = torchaudio.load(wav_path)
#     if not get_duration(wav, sr) > max_duration_sec:
#         valid_lines_valid.append((wav_path, txt))
#     wav = convert_audio(wav, sr, SAMPLE_RATE, CHANNELS).to(device)

#     semantic_vectors = hubert_model.forward(wav, input_sample_hz=SAMPLE_RATE)
#     semantic_tokens = hubert_tokenizer.get_token(semantic_vectors)

#     # save semantic tokens
#     os.makedirs(os.path.join(path, 'tokens'), exist_ok=True)
#     semantic_tokens = semantic_tokens.cpu().numpy()

#     # Extract discrete codes from EnCodec
#     with torch.no_grad():
#         encoded_frames = codec_model.encode(wav.unsqueeze(0))
#     codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1).squeeze()  # [n_q, T]

#     # move codes to cpu
#     codes = codes.cpu().numpy()

#     # save tokens
#     np.savez_compressed(os.path.join(path, 'tokens', os.path.basename(wav_path).replace('.wav', '.npz')), fine=codes, coarse=codes[:2, :], semantic=semantic_tokens)

# # rewrite valid.txt with valid lines
# with open(path + 'valid_valid.txt', 'w', encoding='utf-8') as f:
#     for wav_path, txt in valid_lines_valid:
#         wav_path = os.path.relpath(wav_path, dataset_path).replace('\\', '/')
#         f.write(f'{wav_path}|{txt}\n')

# del hubert_model
# del hubert_tokenizer
# del codec_model
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
model, tokenizer = _load_model(ckpt_path, device, use_small=False, model_type=model_type)


Loaded text model with 312329216 params, val_loss=1.2689.


In [ ]:
if scale_lr:
    learning_rate = (
        learning_rate * grad_accum * train_batch_size * accelerator.num_processes
    )

if use_8bit_adam:
    try:
        import bitsandbytes as bnb
    except ImportError:
        raise ImportError(
            "To use 8-bit Adam, please install the bitsandbytes library: `pip install bitsandbytes`."
        )

    optimizer_class = bnb.optim.AdamW8bit
else:
    optimizer_class = torch.optim.AdamW

In [ ]:
quantization_config=BitsAndBytesConfig(
    load_in_4bit=bits == 4,
    load_in_8bit=bits == 8,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=double_quant,
    bnb_4bit_quant_type=quant_type # {'fp4', 'nf4'}
)

if quantization_config.load_in_8bit or quantization_config.load_in_4bit:
    if quantization_config.load_in_8bit:
        logger.info("Detected 8-bit loading: activating 8-bit loading for this model")
    elif quantization_config.load_in_4bit:
        logger.info("Detected 4-bit loading: activating 4-bit loading for this model")

    # We keep some modules such as the lm_head in their original dtype for numerical stability reasons
    if llm_int8_skip_modules is None or len(llm_int8_skip_modules) == 0:
        modules_to_not_convert = [] # get_keys_to_not_convert(model)
    else:
        modules_to_not_convert = llm_int8_skip_modules

    if not isinstance(modules_to_not_convert, list):
        modules_to_not_convert = [modules_to_not_convert]

    modules_to_not_convert.extend(keep_in_fp32_modules)

    supports_4bit = version.parse(importlib_metadata.version("bitsandbytes")) >= version.parse("0.39.0")

    if quantization_config.load_in_4bit and not supports_4bit:
        raise ValueError(
            "You have a version of `bitsandbytes` that is not compatible with 4bit inference and training"
            " make sure you have the latest version of `bitsandbytes` installed"
        )

    if len(modules_to_not_convert) == 0:
        modules_to_not_convert = None

    model = replace_with_bnb_linear(
        model, modules_to_not_convert=modules_to_not_convert, quantization_config=quantization_config
    )

    # training in 8-bit is only available in 0.37.0+
    model._is_kbit_training_enabled = version.parse(
        importlib_metadata.version("bitsandbytes")
    ) >= version.parse("0.37.0")

    model.config.quantization_config = quantization_config

In [ ]:
if bits == 4:
    from accelerate.utils import CustomDtype
    target_dtype = CustomDtype.INT4
elif bits == 8:
    target_dtype = torch.int8

if lora_dim > 0:
    for param in model.parameters():
        if param.ndim == 1:
            # cast the small parameters (e.g. layernorm) to fp32 for stability
            param.data = param.data.to(torch.float32)

    class CastOutputToFloat(nn.Sequential):
        def forward(self, x):
            return super().forward(x).to(torch.float32)

    model.lm_head = CastOutputToFloat(model.lm_head)

    model = convert_linear_layer_to_lora(model, lora_module_name,
                                            lora_dim=lora_dim, lora_scaling=lora_scaling,
                                            lora_dropout=lora_dropout)
    if optimize_lora_params_only:
        model = only_optimize_lora_parameters(model)

In [ ]:
params_to_optimize = (
        param for param in model.parameters() if param.requires_grad
    )

optimizer = optimizer_class(
    params_to_optimize,
    lr=learning_rate,
    betas=(adam_beta1, adam_beta2),
    weight_decay=weight_decay,
    eps=adam_epsilon,
)

In [ ]:
opt_train = {
    'path': dataset_path,
    'tokenizer': tokenizer,
    'mode': 'train',
}

opt_val = {
    'path': dataset_path,
    'tokenizer': tokenizer,
    'mode': 'valid',
}

train_dataset = TtsDataset(opt_train)
validation_dataset = TtsDataset(opt_val)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    collate_fn=TtsCollater(),
)

validation_dataloader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=eval_batch_size,
    collate_fn=TtsCollater(),
)

criterion = torch.nn.CrossEntropyLoss() #ignore_index=SEMANTIC_PAD_TOKEN)

# Scheduler and math around the number of training steps.
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / grad_accum)
if max_train_steps is None:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True

lr_scheduler = get_scheduler(
    lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=lr_warmup_steps * grad_accum,
    num_training_steps=max_train_steps * grad_accum,
)

In [ ]:
model, optimizer, train_dataloader, validation_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, validation_dataloader, lr_scheduler
)
accelerator.register_for_checkpointing(lr_scheduler)

weight_dtype = torch.float32
if accelerator.mixed_precision == "fp16":
    weight_dtype = torch.float16
elif accelerator.mixed_precision == "bf16":
    weight_dtype = torch.bfloat16

In [ ]:
# We need to recalculate our total training steps as the size of the training dataloader may have changed.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / grad_accum)
if overrode_max_train_steps:
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
# Afterwards we recalculate our number of training epochs
num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

# We need to initialize the trackers we use, and also store our configuration.
# The trackers initializes automatically on the main process.
if accelerator.is_main_process:
    accelerator.init_trackers("bark_semantic", config={})

# Train!
total_batch_size = train_batch_size * accelerator.num_processes * grad_accum
logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num batches each epoch = {len(train_dataloader)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {grad_accum}")
logger.info(f"  Total optimization steps = {max_train_steps}")
global_step = 0
first_epoch = 0

if resume_from_checkpoint:
    if resume_from_checkpoint != "latest":
        path = os.path.basename(resume_from_checkpoint)
    else:
        # Get the most recent checkpoint
        dirs = os.listdir(output_dir)
        dirs = [d for d in dirs if d.startswith("checkpoint")]
        dirs = sorted(dirs, key=lambda x: int(x.split("-")[1]))
        path = dirs[-1]
    accelerator.print(f"Resuming from checkpoint {path}")
    accelerator.load_state(os.path.join(output_dir, path))
    global_step = int(path.split("-")[1])

    resume_global_step = global_step * grad_accum
    first_epoch = resume_global_step // num_update_steps_per_epoch
    resume_step = resume_global_step % num_update_steps_per_epoch

In [ ]:
print(num_train_epochs)

15


In [ ]:
# !pip3 uninstall torch torchvision torchaudio
# !pip3 install torch torchvision torchaudio


In [ ]:
# os.environ['CUDA_VISIBLE_DEVICES'] ='0'
torch.cuda.empty_cache()


In [ ]:
if accelerator.is_main_process:
    model.eval()
    validation_loss = 0.0
    num_batches = 0
    num_samples = 0
    with torch.no_grad():
        for val_step, val_batch in enumerate(validation_dataloader):
            # Similar to training, process the validation batch
            val_targets = val_batch['semantic_tokens'][:, 1:].contiguous()
            val_semantic_inputs = val_batch['semantic_tokens'][:, :-1]
            val_inputs = torch.cat([val_batch['input_ids'], val_semantic_inputs], dim=1)

            # Forward pass for validation
            val_logits = model(val_inputs, training=True)
            val_semantic_logits = val_logits[:, val_batch['input_ids'].size(1):].contiguous()

            # Calculate the validation loss
            val_loss = criterion(val_semantic_logits.view(-1, model.config.output_vocab_size), val_targets.view(-1))
            validation_loss += val_loss.item()
            num_batches += 1
            num_samples += val_batch['input_ids'].size(0)

    average_validation_loss = validation_loss / num_batches
    logger.info(f"Validation Loss: {average_validation_loss} over {num_samples} samples and {num_batches} batches.")
    print(f"Validation Loss: {average_validation_loss} over {num_samples} samples and {num_batches} batches.")

Validation Loss: 0.8708278409498078 over 2099 samples and 700 batches.


In [ ]:
# Only show the progress bar once on each machine.
progress_bar = tqdm(range(global_step, max_train_steps), disable=not accelerator.is_local_main_process)
progress_bar.set_description("Steps")

for epoch in range(first_epoch, num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Skip steps until we reach the resumed step
        if resume_from_checkpoint and epoch == first_epoch and step < resume_step:
            if step % grad_accum == 0:
                progress_bar.update(1)
            continue

        with accelerator.accumulate(model):
            targets = batch['semantic_tokens'][:, 1:].contiguous()

            # Remove the last semantic token from the inputs since there is no target for it.
            semantic_inputs = batch['semantic_tokens'][:, :-1]

            # Combine the text and semantic tokens and feed them into the model.
            inputs = torch.cat([batch['input_ids'], semantic_inputs], dim=1)
            logits = model(inputs, training=True)

            # We're only interested in the logits for the semantic tokens, so we ignore the logits for the input text tokens.
            semantic_logits = logits[:, batch['input_ids'].size(1):].contiguous()

            # Compute the loss.
            loss = criterion(semantic_logits.view(-1, model.config.output_vocab_size), targets.view(-1))

            accelerator.backward(loss)
            if accelerator.sync_gradients:
                params_to_clip = (
                    param for param in model.parameters() if param.requires_grad
                )
                accelerator.clip_grad_norm_(params_to_clip, max_grad_norm)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        # Checks if the accelerator has performed an optimization step behind the scenes
        if accelerator.sync_gradients:
            progress_bar.update(1)
            global_step += 1

            if global_step % checkpointing_steps == 0:
                if accelerator.is_main_process:
                    save_path = os.path.join(output_dir, f"checkpoint-{global_step}")
                    accelerator.save_state(save_path)
                    logger.info(f"Saved state to {save_path}")

        logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0]}
        progress_bar.set_postfix(**logs)
        accelerator.log(logs, step=global_step)

        if global_step >= max_train_steps:
            break

    accelerator.wait_for_everyone()

if accelerator.is_main_process:
    if lora_dim > 0:
        model = convert_lora_to_linear_layer(model)
    # save model
    accelerator.save(model.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))

    config = model.config.__dict__
    # save config
    with open(os.path.join(output_dir, "config.json"), "w") as f:
        json.dump(config, f, indent=2)

accelerator.end_training()

  0%|          | 0/8175 [00:00<?, ?it/s]

In [ ]:
if accelerator.is_main_process:
    model.eval()
    validation_loss = 0.0
    num_batches = 0
    num_samples = 0
    with torch.no_grad():
        for val_step, val_batch in enumerate(validation_dataloader):
            # Similar to training, process the validation batch
            val_targets = val_batch['semantic_tokens'][:, 1:].contiguous()
            val_semantic_inputs = val_batch['semantic_tokens'][:, :-1]
            val_inputs = torch.cat([val_batch['input_ids'], val_semantic_inputs], dim=1)

            # Forward pass for validation
            val_logits = model(val_inputs, training=True)
            val_semantic_logits = val_logits[:, val_batch['input_ids'].size(1):].contiguous()

            # Calculate the validation loss
            val_loss = criterion(val_semantic_logits.view(-1, model.config.output_vocab_size), val_targets.view(-1))
            validation_loss += val_loss.item()
            num_batches += 1
            num_samples += val_batch['input_ids'].size(0)

    average_validation_loss = validation_loss / num_batches
    logger.info(f"Validation Loss: {average_validation_loss} over {num_samples} samples and {num_batches} batches.")
    print(f"Validation Loss: {average_validation_loss} over {num_samples} samples and {num_batches} batches.")

In [ ]:
# !mv "/content/bark-with-voice-clone/semantic_output" "/content/drive/Shareddrives/AIR PH/RVC/jonathan-folder/OCT6/2/epoch_55"


In [ ]:
# !mv "/content/bark-with-voice-clone/semantic_output" "/content/drive/Shareddrives/AIR PH/RVC/jonathan-folder/OCT2/preprocess/models_1"


In [ ]:
import os
import time
from google.colab import drive

# Define the source and destination paths
source_path = "/content/bark-with-voice-clone/semantic_output"
destination_path = "/content/drive/MyDrive/jonathan-folder/OCT16/models/epoch15"

# Move the file
!mv "$source_path" "$destination_path"

# Check if the move operation was successful
if os.path.exists(destination_path):
    print("File moved successfully.")
else:
    print("File move failed.")

# Define the maximum runtime (in seconds) before disconnecting
max_runtime_seconds = 20 * 60  # 20 minutes

# Get the current time
start_time = time.time()

# Check the runtime and disconnect if necessary
while True:
    current_time = time.time()
    elapsed_time_seconds = current_time - start_time

    # Disconnect if the file was moved successfully or if the maximum runtime is reached
    if os.path.exists(destination_path) or elapsed_time_seconds >= max_runtime_seconds:
        print("Disconnecting runtime...")
        drive.flush_and_unmount()
        os.system("pkill -9 -f ipykernel_launcher")
        break

    # Sleep for a short duration before checking again
    time.sleep(60)  # Check every minute


In [ ]:
from google.colab import runtime
runtime.unassign()